In [1]:
import pandas as pd

# Задание 1 
Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.
Результат классификации запишите в отдельный столбец region.

In [2]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [4]:
data_key = pd.read_csv('keywords.csv')
data_key.head(2)

,keyword,shows
0,вк,64292779
1,одноклассники,63810309


In [6]:
def add_region(row):
    geo_list = []
    for i in geo_data['Центр']:
        if i in row['keyword']:
            return 'Центр'
    for i in geo_data['Северо-Запад']:
        if i in row['keyword']:
            return 'Северо-Запад'
    for i in geo_data['Дальний Восток']:
        if i in row['keyword']:
            return 'Дальний Восток'
        return 'undefined'

In [7]:
data_key['region'] = data_key.apply(add_region, axis=1)

In [9]:
data_key.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [8]:
data_key[data_key['region'] != 'undefined'].head() #теперь посмотрим топ запросов

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток


# Задание 2 
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
оценка 2 и меньше - низкий рейтинг
оценка 4 и меньше - средний рейтинг
оценка 4.5 и 5 - высокий рейтинг
Результат классификации запишите в столбец class

In [10]:
movies = pd.read_csv('Movies_small/ratings.csv')

In [11]:
movies.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [35]:
data_movies = movies.groupby('movieId').agg(['mean'])['rating'].reset_index()
data_movies.head(3) #сначала найдем среднюю оценку для каждого фильма

,movieId,mean
0,1,3.872470
1,2,3.401869
2,3,3.161017


In [32]:
def class_marks(data):
    if data['mean'] <= 2.0: 
         return 'Низкий рейтинг'
    elif data['mean'] > 2.0 and data['mean'] <= 4.0:
        return 'Средний рейтинг'
    elif data['mean'] > 4.0:
        return 'Высокий рейтинг'

In [33]:
data_movies['mark_rating'] = data_movies.apply(class_marks, axis=1)

In [34]:
data_movies[data_movies['mark_rating'] == 'Высокий рейтинг'].\
    sort_values(by='mean', ascending=False).head()

,movieId,mean,mark_rating
9065,163949,5.0,Высокий рейтинг
2544,3165,5.0,Высокий рейтинг
2504,3112,5.0,Высокий рейтинг
6330,43267,5.0,Высокий рейтинг
6490,47728,5.0,Высокий рейтинг


# Задание 3 
Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). 
Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [47]:
import time
import datetime
from datetime import date
from datetime import datetime

In [37]:
movies.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [38]:
data_list = movies.groupby(['userId']).count().reset_index()[['userId', 'movieId']]

In [39]:
data_list.head(3)

,userId,movieId
0,1,20
1,2,76
2,3,51


In [40]:
loyal_users_100 = data_list[data_list['movieId'] >= 100] #сделаем объект только с лояльными юзерами

In [41]:
loyal_users_100.head(3)

,userId,movieId
3,4,204
4,5,100
7,8,116


In [42]:
loyal_users_100_list = loyal_users_100['userId'].tolist() #переведем в список

In [43]:
loyal_users_100_list[0] #проверим

4

In [51]:
only_fans_data = movies[movies['userId'].isin(loyal_users_100_list)] #сделаем так, чтобы в таблице были лишь лояльные
only_fans_data.head()

,userId,movieId,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582
150,4,141,5.0,949919681
151,4,153,4.0,949811346


In [52]:
only_fans_data['timestamp'] = only_fans_data['timestamp'].apply(lambda x: datetime.fromtimestamp(x))

/Users/staroverova/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [53]:
only_fans_data.head() #проеверим как перевелось

,userId,movieId,rating,timestamp
147,4,10,4.0,2000-02-06 07:17:25
148,4,34,5.0,2000-02-07 13:32:36
149,4,112,5.0,2000-02-06 07:16:22
150,4,141,5.0,2000-02-07 13:34:41
151,4,153,4.0,2000-02-06 07:29:06


In [54]:
only_fans_data = only_fans_data.groupby('userId').agg(['min', 'max'])['timestamp'].reset_index() 

In [56]:
only_fans_data.insert(3, 'Lifetime', (only_fans_data['max'] - only_fans_data['min']))

In [57]:
only_fans_data.head()

,userId,min,max,Lifetime
0,4,2000-02-05 22:25:14,2000-02-08 06:57:54,2 days 08:32:40
1,5,2006-11-13 02:10:44,2006-11-13 02:45:45,0 days 00:35:01
2,8,2006-08-01 03:42:20,2006-08-02 03:22:07,0 days 23:39:47
3,15,2001-08-16 08:47:19,2016-07-24 06:25:35,5455 days 21:38:16
4,17,2005-09-23 13:43:07,2005-09-23 15:57:20,0 days 02:14:13


In [58]:
only_fans_data[['userId', 'Lifetime']].sort_values(by='Lifetime', ascending=False).head() #топ-5 самых лояльных

,userId,Lifetime
140,380,5969 days 18:54:21
212,547,5807 days 23:48:55
161,427,5515 days 19:20:56
3,15,5455 days 21:38:16
248,624,5295 days 00:15:07
